In [1]:
# -*- coding: UTF-8 -*-
import sys

try:
    from math import exp, sqrt
    import os
    import arcpy
    arcpy.env.overwriteOutput = True
    import networkx as nx
    import time
    import matplotlib.pyplot as plt
    import numpy

except ImportError:
    print "Error: missing one of the libraries (arcpy, networkx)"
    sys.exit()

In [2]:
track = "testTrack.shp"
segments = "testSegments.shp"
trackpoints = []

In [3]:
if arcpy.Exists(track):
    #print "yes"
    for row in arcpy.da.SearchCursor(track,["SHAPE@"]):
        #搜索出来的就是点要素
        #print row[0]
        trackpoints.append(row[0])
print trackpoints

[<PointGeometry object at 0x1e3523d0[0x1e352400]>, <PointGeometry object at 0x565e670[0x61f10c0]>, <PointGeometry object at 0x1e35ecb0[0x1e35ec80]>, <PointGeometry object at 0x1e35ecf0[0x1e35ecc0]>, <PointGeometry object at 0x1e35ed30[0x1e35ed00]>, <PointGeometry object at 0x1e35ed70[0x1e35ed40]>, <PointGeometry object at 0x1e35edb0[0x1e35ed80]>, <PointGeometry object at 0x1e35edf0[0x1e35edc0]>, <PointGeometry object at 0x1e35ee30[0x1e35ee00]>, <PointGeometry object at 0x1e35ee70[0x1e35ee40]>, <PointGeometry object at 0x1e35eeb0[0x1e35ee80]>, <PointGeometry object at 0x1e35eef0[0x1e35eec0]>, <PointGeometry object at 0x1e35ef30[0x1e35ef00]>, <PointGeometry object at 0x1e35ef70[0x1e35ef40]>]


In [4]:
if arcpy.Exists(segments):
    cursor = arcpy.da.SearchCursor(segments,["OBJECTID","SHAPE@"])  
    #这样搜索出来的每个结果就是两个，一个是objectid一个是shape@，具体看下面的结果
    #row[0]就是objectid，row[1]就是对应的线要素
    endpoint = {}
    segmentlengths = {}
    for row in cursor:
        #print row
        endpoint[row[0]]=((row[1].firstPoint.X,row[1].firstPoint.Y), (row[1].lastPoint.X, row[1].lastPoint.Y))
        #上面这一句就是获取每个线段的起终点
        segmentlengths[row[0]]=row[1].length
        #获取每个线段的长度
print endpoint
print segmentlengths

{1513476: ((160664.3365626894, 385910.50146254525), (160660.52330773324, 385930.7246584892)), 1513477: ((160700.38363555074, 386438.48022403196), (160700.36310744286, 386456.68494686484)), 1513478: ((160714.31991149113, 386404.10919384286), (160700.38363555074, 386438.48022403196)), 1513479: ((160569.14611157402, 386296.741236303), (160456.75941053033, 386242.0044410974)), 1513480: ((160700.36310744286, 386456.68494686484), (160718.02091970295, 386483.0006448515)), 1513481: ((160630.25275033712, 385882.14481842145), (160641.69906043634, 385815.4068011753)), 1513482: ((160431.82297675684, 385932.4964925423), (160523.99179366603, 385995.30168914795)), 1513483: ((160457.13496907428, 385894.09154822305), (160574.65708200634, 385882.08321109414)), 1513484: ((160565.1219664365, 386509.125860665), (160579.0571309775, 386474.754357215)), 1520642: ((161642.03302863985, 386122.0574251935), (161629.39821569622, 386122.04084613547)), 1513489: ((160292.4751143679, 386276.2165215127), (160337.929034

In [5]:
#print arcpy.env.workspace

In [6]:
#if arcpy.Exists(segments):
#    print "yes"

In [7]:
g = nx.read_shp(segments)   #g就是一个图，就是提供的shp形成的一个图
g #g是digraph对象，是有向图

In [8]:
sg = list(nx.connected_component_subgraphs(g.to_undirected()))[0]  #这个操作是干嘛的还没看懂
#.to_undirected是转换成无向图
#获取连通分量(nx.connected_component_subgraphs(G)，返回的是列表，但是元素是图，这些分量按照节点数目从大到小排列，所以第一个就是最大的连通分量)。
#获取的是一个由图组成的数组，所以sg就是第0个元素，也就是最大连通分量的元素
sg #sg是graph对象，是无向图

In [9]:
print len(g)
print len(sg)  #为什么两个print出来的数量不一样
#大概明白了，原图不是一个连通图，有些游离于大部队之外，这里只提取其中的最大联通部分，也就是最大连通分量元素
#可以尝试写出来，放到gis里面对比一下
print g.number_of_nodes()
print g.number_of_edges()
print sg.number_of_nodes()
print sg.number_of_edges()

573
520
573
735
520
681


In [10]:
print "graph size (excluding unconnected parts): "+str(len(g))

graph size (excluding unconnected parts): 573


In [11]:
#把路段长度赋值给图sg的边的length属性
for n0, n1 in sg.edges():
    oid = sg[n0][n1]["OBJECTID"]
    sg[n0][n1]['length'] = segmentlengths[oid]
    

In [20]:
point = trackpoints[2]
p=point.firstPoint
p

<Point (160989.864492, 386224.541953, #, #)>

In [21]:
print type(p)
print p.X
print p.Y

<class 'arcpy.arcobjects.arcobjects.Point'>
160989.864492
386224.541953


In [22]:
maxdist = 50

In [15]:
arcpy.MakeFeatureLayer_management(segments, "segments_lyr")
#因为不是从arctoolbox启动，这个图层没有，所以要创建一个


<Result 'segments_lyr'>

In [23]:
arcpy.SelectLayerByLocation_management ("segments_lyr", "WITHIN_A_DISTANCE", point, maxdist,"NEW_SELECTION")

<Result 'segments_lyr'>

In [24]:
candidates = {}
#Go through these, compute distances, probabilities and store them as candidates
cursor = arcpy.da.SearchCursor('segments_lyr', ["OBJECTID", "SHAPE@"])
#row =[]
for row in cursor:
    print row[0]  #好奇怪，这里第一个点trackpoints[0]只选择了一个出来，其实应该选出来三个的

1513627
1513628
1513629
1513630
1513631
1513639
1513642
1513643
1513644
1513645


In [ ]:
for row in cursor:
    feat = row[1]
    #compute the spatial distance
    dist = point.distanceTo(row[1])
    #compute the corresponding probability
    candidates[row[0]] = getPDProbability(dist, decayConstantEu)
del row
del cursor